In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from plotting_tools import * 

In [58]:
import scipy.stats as stats

In [59]:
experiment_series ="pop8"
precipitation_setting = "Rain"
decay_setting = "decay"

In [60]:
result_path = f"../../preprocessing/preprocessed_data/{experiment_series}"

df_simulations = pd.read_csv(f"{result_path}/substances/{decay_setting}_{precipitation_setting}_output.csv")
df_simulations = df_simulations.loc[df_simulations["variable"]=="COV19"]
df_simulations["Location"] = df_simulations.manhole.apply(lambda x: manhole_names[x] if x in manhole_names else x)
df_simulations["Date"] = pd.to_datetime(start_date) + pd.to_timedelta(df_simulations["time_in_minutes"], unit="min")

df_measurements = pd.read_csv(f"{result_path}/../concentration_measurements.csv")

# df_simulations = df_simulations.loc[df_simulations["Location"].isin(df_measurements["Location"].unique())]

In [61]:
df_measurements["sampling_timepoint"] = pd.to_datetime(df_measurements.Date)
df_measurements.rename(columns={"Value": "value"}, inplace=True)

In [62]:
df_measurements = df_measurements.loc[df_measurements.value>75]

In [63]:
df_simulations.Hour = df_simulations.Date.dt.hour
df_simulations = df_simulations.loc[(df_simulations.Hour<=11)&(df_simulations.Hour>=9)]
df_simulations.Date = pd.to_datetime(df_simulations.Date.dt.date)

/tmp/ipykernel_15658/906119272.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_simulations.Hour = df_simulations.Date.dt.hour


In [64]:
df_simulations

,time_in_minutes,variable,value,manhole,time_in_days,simulation_id,Location,Date
35,540,COV19,0.00000,MUC012,0.375000,1,N_Uc,2020-03-02
36,555,COV19,0.00000,MUC012,0.385417,1,N_Uc,2020-03-02
37,570,COV19,0.00000,MUC012,0.395833,1,N_Uc,2020-03-02
38,585,COV19,0.00000,MUC012,0.406250,1,N_Uc,2020-03-02
39,600,COV19,0.00000,MUC012,0.416667,1,N_Uc,2020-03-02
...,...,...,...,...,...,...,...,...
30023312,131685,COV19,219.94832,MUC616,91.447917,100,S_M1,2020-06-01
30023313,131700,COV19,221.27918,MUC616,91.458333,100,S_M1,2020-06-01
30023314,131715,COV19,222.74467,MUC616,91.468750,100,S_M1,2020-06-01
30023315,131730,COV19,224.24210,MUC616,91.479167,100,S_M1,2020-06-01


In [65]:
df_res = pd.merge(df_simulations, df_measurements, how="left", left_on=["Date"], right_on=["sampling_timepoint"], suffixes=("_sim", "_meas"))

In [66]:
df_res = df_res.loc[df_res.value_meas.notna(), ["Location_sim", "Location_meas", "simulation_id", "time_in_minutes", "value_meas", "value_sim"]]

In [76]:
df_res = df_res.loc[df_res.value_sim!=0]

In [77]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

meas = df_res['value_meas'].values
sim  = df_res['value_sim'].values

def neg_log_likelihood(params, meas, sim):
    """
    Negative log-likelihood for:
      meas_i ~ LogNormal(log(k*sim_i), sigma^2)
    params = [ln_k, ln_sigma]
    """
    ln_k, ln_sigma = params
    sigma = np.exp(ln_sigma)
    mu = ln_k + np.log(sim)
    # drop the -ln(x) term since it doesn't depend on k or sigma
    resid = (np.log(meas) - mu)
    # NLL up to an additive constant:
    return 0.5 * np.sum((resid/sigma)**2 + 2*ln_sigma)

# initial guesses: use simple log‐ratio MLE for ln_k, and its std for ln_sigma
initial_ln_k = 1.0
initial_ln_sigma = np.log(np.log(meas/sim).std(ddof=0))
x0 = np.array([initial_ln_k, initial_ln_sigma])

# enforce sigma>0 but ln_sigma free; k>0 automatically via ln_k
res = minimize(neg_log_likelihood, x0,
               args=(meas, sim),
               bounds=[(None, None), (None, None)])

ln_k_opt, ln_sigma_opt = res.x
k_opt = np.exp(ln_k_opt)
sigma_opt = np.exp(ln_sigma_opt)

print(f"  Estimated scaling k = {k_opt:.6f}")
print(f"Estimated log‐space σ = {sigma_opt:.6f}")


  Estimated scaling k = 1.100993
Estimated log‐space σ = 1.436032


## Apply scaling

In [80]:
for setting in ["decay_Rain", "no_decay_noRain", "no_decay_Rain"]:
    result_path = f"../../preprocessing/preprocessed_data/pop8"
    df_simulations = pd.read_csv(f"{result_path}/substances/{setting}_output.csv")
    df_simulations.value = df_simulations.value * k_opt
    df_simulations.to_csv(f"{result_path}/substances/{setting}_output_scaled.csv", index=False)


result_path = f"../../preprocessing/preprocessed_data/pop8_local"
setting = "decay_Rain"
df_simulations = pd.read_csv(f"{result_path}/substances/{setting}_output.csv")
df_simulations.value = df_simulations.value * k_opt
df_simulations.to_csv(f"{result_path}/substances/{setting}_output_scaled.csv", index=False)